In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#Basic PyTorch setup
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#Use GPU if available
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
data = pd.read_csv("10p_out.csv")
data["tag_name"] = data["tag_name"].astype('category')
data.tag_name = data.tag_name.cat.codes
data

x_data = data.copy()
y_data = data["tag_resp"]
del x_data["tag_resp"]

# For the two most important features
del x_data["#_in_feedback"]
del x_data["prob_threshold"]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05, random_state=42)

In [6]:
class MLP(nn.Module):
    def __init__(self, input_dim = 2, hid_dim = 2, output_dim=1):
        # initialze the superclass
        super(MLP, self).__init__()
        # this will create a linear layer with input_dim x hid_dim parameters
        # in addition to a bias unit with hid_dim parameters
        self.lin1 = nn.Linear(input_dim, hid_dim)
        # same thing here except hid_dim x output_dim
        self.lin2 = nn.Linear(hid_dim, output_dim)
    
    def forward(self, x):
        x = self.lin1(x)  # linear combination of inputs
        x = torch.sigmoid(x) # then through sigmoid - output of first layer
        x = self.lin2(x) # linear combination of hidden units
        x = torch.sigmoid(x) # then through sigmoid - net output
        return x

#Initialize weights of this model
def weights_init(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            # initialize the weight tensor, here we use a normal distribution
            m.weight.data.normal_(0, 1)

model = MLP().to(dev)
weights_init(model)

X = torch.Tensor(x_train.to_numpy()).to(dev)
Y = torch.Tensor(y_train.to_numpy()).view(-1,1).to(dev)
#loss_func = nn.CrossEntropyLoss().to(dev)
loss_func = nn.BCEWithLogitsLoss().to(dev)
#optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 51
N = X.size(0)
for i in range(epochs):
  order=np.random.permutation(N) #Generate a random ordering (this ensures every datapoint is used for training, just in random order)
  for j in range(N):
    data_point = order[j]
    x_var = Variable(X[data_point], requires_grad=False)
    y_var = Variable(Y[data_point], requires_grad=False)
    optimizer.zero_grad()
    y_hat = model(x_var) #Calculate the output of the model
    loss = loss_func.forward(y_hat, y_var) #Calculate loss between model output and target values
    loss.backward() #Back propogation
    optimizer.step() #Use the gradients to update weights, min loss

  if i % 10 == 0:
    print("Epoch: {0}, Loss: {1}, ".format(i, loss.data.cpu().numpy()))

list(model.parameters())

Epoch: 0, Loss: 0.6931145191192627, 
Epoch: 10, Loss: 0.6931471824645996, 
Epoch: 20, Loss: 0.6931471824645996, 
Epoch: 30, Loss: 0.6931471824645996, 
Epoch: 40, Loss: 0.6931471824645996, 
Epoch: 50, Loss: 0.6931471824645996, 


[Parameter containing:
 tensor([[ 0.7357, -0.2004],
         [ 1.0651,  1.3090]], requires_grad=True), Parameter containing:
 tensor([3.7176, 4.3277], requires_grad=True), Parameter containing:
 tensor([[-9.0815, -7.7403]], requires_grad=True), Parameter containing:
 tensor([-7.5777], requires_grad=True)]

In [ ]:
model.eval()
x_test_tensor = torch.Tensor(x_test.to_numpy()).to(dev)
with torch.no_grad():
    y_pred = model(x_test_tensor)

count = 0
correct = 0
for test_val in y_test:
  pred_val = 1 if y_pred[i] > 0.5 else 0
  if pred_val == test_val:
    correct += 1
  count += 1

print("Accuracy: {}".format(correct/count))